**Problem 1 (50 pts).** This problem will involve the nycflights13 dataset (including the tables
flights, airlines, airports, planes and weather), which we saw in class. It is available in both R and
Python, however R is recommended for at least the visualization portion of the question. You can
get more information about this package on github at

https://github.com/tidyverse/nycflights13

You can access the data tables in R by loading the nycflights13 library using the following:

library(nycflights13)

Alternatively, you can download the csv files from the Modules page on Canvas. The files are
flights.csv, weather.csv, planes.csv, airports.csv, airlines.csv.

We will first use joins to search and manipulate the dataset, then we will produce a flight count
visualization.

**a. (10 pts)** Filter the dataset (using a left join) to display the tail number, year, month,
day, hour, origin, and humidity for all flights heading to Tampa International
Airport (TPA) after 12pm on November 1, 2013. How many flights happened
during the given time frame that day?


In [ ]:
import pandas as pd

flights = pd.read_csv('flights.csv')
weather = pd.read_csv('weather.csv')
planes = pd.read_csv('planes.csv')
airports = pd.read_csv('airports.csv')
airlines = pd.read_csv('airlines.csv')

# print(weather.head())
# print(flights.head())

# Corresponds to year, month, day and hour.
weather['datetime'] = pd.to_datetime(weather['time_hour'])
weather = weather.assign(
    year=weather['datetime'].dt.year,
    month=weather['datetime'].dt.month,
    day=weather['datetime'].dt.day,
    hour=weather['datetime'].dt.hour
)

# According to dest(TPA) and time to filter.
filter_flights = flights[
    (flights['tailnum'].notna()) &
    (flights['dest'] == 'TPA') &
    (flights['year'] == 2013) &
    (flights['month'] == 11) &
    (flights['day'] == 1) &
    (flights['dep_time'] > 1200)
]

# Use left join.
filtered_data = pd.merge(
    # Left is flights data.
    left=filter_flights,
    # Rights is weathers data.
    right=weather[['year', 'month', 'day', 'hour', 'origin', 'humid']],
    # Merge those make data match,
    on=['year', 'month', 'day', 'hour', 'origin'],
    how='left'
)

# Display the tail number, year, month, day, hour, origin, and humidity for all flights heading to TPA after 12pm on November 1, 2013.
flights_weather_results = filtered_data[['tailnum', 'year', 'month', 'day', 'hour', 'origin', 'humid']]

# Print the result and the number of flights
print(flights_weather_results)
print("-----------------------------------------")
print(f"{len(flights_weather_results)} flights happened during the given time frame that day.")


  tailnum  year  month  day  hour origin  humid
0  N580JB  2013     11    1    14    JFK  88.09
1  N337NB  2013     11    1    14    LGA  72.74
2  N567UA  2013     11    1    15    EWR  80.49
3  N515MQ  2013     11    1    14    JFK  88.09
4  N779JB  2013     11    1    15    EWR  80.49
5  N561JB  2013     11    1    16    LGA  67.57
6  N974DL  2013     11    1    18    JFK  63.08
7  N319NB  2013     11    1    19    LGA  50.60
8  N76265  2013     11    1    19    EWR  52.80
9  N768JB  2013     11    1    19    JFK  65.07
-----------------------------------------
10 flights happened during the given time frame that day.


**b. (10 pts)** What is the difference between the following two joins?

anti_join(flights, airports, by = c("dest" = "faa"))

anti_join(airports, flights, by = c("faa" = "dest"))



**Answer**

First anti_join returns the "dest" of the flights where it cannot find a match in the airports("faa").

First anti_join returns the "faa" of the airports where it cannot find a match in the flights("dest").

**c. (10 pts)** Select the origin and destination airports and their latitude and longitude
for all flights in the dataset (using one or more inner joins). How many flights are
there in your result?

In [ ]:
import pandas as pd

# Select the origin and destination airports.
origin_data = pd.merge(flights, airports[['faa', 'lat', 'lon']], left_on='origin', right_on='faa', how='inner')
origin_data = origin_data.rename(columns={'lat': 'origin latitude', 'lon': 'origin longitude'})
# Use inner, only data ​​that exist in both tables will be merged.
lat_lon_data = pd.merge(origin_data, airports[['faa', 'lat', 'lon']], left_on='dest', right_on='faa', how='inner')
lat_lon_data = lat_lon_data.rename(columns={'lat': 'dest latitude', 'lon': 'dest longitude'})
# Get the all flights's origin and destination airports and their latitude and longitude.
lat_lon_data = lat_lon_data[['origin', 'origin latitude', 'origin longitude', 'dest', 'dest latitude', 'dest longitude']]

print(lat_lon_data)
print("------------------------------------------------")
print(f"There are {len(lat_lon_data)} flights.")


       origin  origin latitude  origin longitude dest  dest latitude  \
0         EWR        40.692500        -74.168667  IAH      29.984433   
1         EWR        40.692500        -74.168667  IAH      29.984433   
2         EWR        40.692500        -74.168667  IAH      29.984433   
3         EWR        40.692500        -74.168667  IAH      29.984433   
4         EWR        40.692500        -74.168667  IAH      29.984433   
...       ...              ...               ...  ...            ...   
329169    JFK        40.639751        -73.778925  MVY      41.391667   
329170    JFK        40.639751        -73.778925  MVY      41.391667   
329171    JFK        40.639751        -73.778925  MVY      41.391667   
329172    JFK        40.639751        -73.778925  MVY      41.391667   
329173    JFK        40.639751        -73.778925  MVY      41.391667   

        dest longitude  
0           -95.341442  
1           -95.341442  
2           -95.341442  
3           -95.341442  
4         

**d. (10 pts)** Use group_by and count to get the number of flights to each unique
carrier/dest combination. How many unique combination of carrier/dest are
present?

In [ ]:
# print(flights.head())

# Use size().reset_index() method.
unique_com = flights.groupby(['carrier','dest']).size().reset_index(name='counts')
print(unique_com)
counts_com = len(unique_com)
print("------------------------------------------------------")
print("Number of unique combination of carrier/dest is:", counts_com)

    carrier dest  counts
0        9E  ATL      59
1        9E  AUS       2
2        9E  AVL      10
3        9E  BGR       1
4        9E  BNA     474
..      ...  ...     ...
309      WN  PHX     431
310      WN  STL    1469
311      YV  CLT     282
312      YV  IAD     311
313      YV  PHL       8

[314 rows x 3 columns]
------------------------------------------------------
Number of unique combination of carrier/dest is: 314


**e. (10 pts)** Produce a map that sizes each destination airport by the number of
incoming flights. You may use a continuous scale for the size. Here is a code
snippet to draw a map of all flight destinations, which you can use as a starting point. You may need to install the maps packages if you have not already. Adjust
the title, axis labels and aesthetics to make this visualization as clear as possible.

flights %>%

 left_join(airports, c("dest" = "faa")) %>%

 ggplot(aes(lon, lat)) +

 borders("state") +

 geom_point() +

 coord_quickmap()

In [ ]:
!pip install plotly
import plotly.express as px

In [ ]:
# Count the number of flights at the destination.
dest_traffic= flights.groupby('dest').size().reset_index(name='dest traffic')

# Merge with airport data to get lat and lon
location = pd.merge(dest_traffic, airports[['faa', 'lat', 'lon']], left_on='dest', right_on='faa')

map_show = px.scatter_geo(
    scope='usa',
    data_frame=location,
    title="Each destination airport by the number of incoming flights in the U.S.",
    lat='lat',
    lon='lon',
    size='dest traffic',
    size_max=10,
    # text=location['dest traffic'],
    color='dest traffic'
    # color_discrete_sequence=['red']
)
map_show.update_layout(width=800, height=800)
map_show.show()